# Investigation - Office 365 Suspicious Logins
### Description:

# Setup

### Install Packages

Uncomment anything you need to install (first time uncomment everything). This will install these Python libraries into your environent so only needed once per machine (or install).

In [ ]:
# You may needs these - should only need to uncomment and run once
!pip install --user --upgrade pip
!pip install Kqlmagic --no-cache-dir --upgrade
!pip install maxminddb-geolite2
!pip install folium
# # May not need this if KqlMagic is installed
#!pip install azure.kusto.data --upgrade

### Imports and Magic

In [ ]:
# Javascript control to hide/display code cells.
# 'toggle_code()' somewhere in a cell to use this
from IPython.core.display import display, HTML
toggle_code_str = '''
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show/Hide Code"></form>
'''

toggle_code_prepare_str = '''
    <script>
    function code_toggle() {
        if ($('div.cell.code_cell.rendered.selected div.input').css('display')!='none'){
            $('div.cell.code_cell.rendered.selected div.input').hide();
        } else {
            $('div.cell.code_cell.rendered.selected div.input').show();
        }
    }
    </script>

'''

display(HTML(toggle_code_prepare_str + toggle_code_str))

def toggle_code():
    display(HTML(toggle_code_str))

In [ ]:
#Imports
from azure.kusto.data.request import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError

from IPython import get_ipython
from IPython.display import display
from ipywidgets import interactive, Layout, Select, IntSlider, widgets, Button, HBox, VBox
from geolite2 import geolite2
import folium
from folium.plugins import MarkerCluster

import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

# KqlMagic
%reload_ext Kqlmagic
%config Kqlmagic.auto_dataframe=True
toggle_code()

### KqlMagic
Documentation https://github.com/mbnshtck/jupyter-kql-magic/blob/master/README.rst

Quickstart Notebook https://github.com/mbnshtck/jupyter-kql-magic/blob/master/notebooks/QuickStart.ipynb

In [ ]:
# Kusto utility functions
# Allows printing of query with stripped commments and newline chars
def print_kql(query):
    print(clean_kql_query(query))

def clean_kql_query(query_string):
    import re
    return re.sub(r'(//[^\"\'\n]+)', '', query_string, re.MULTILINE).replace('\n', '').strip()

def execute_kql_query(query_string):
    if not query_string or len(query_string.strip()) == 0:
        print('No query supplied')
        return None
    src_query = clean_kql_query(query_string)
    magic_string = f'results_frame << {src_query}'
    magic_string = magic_string
    ip = get_ipython()
    ip.run_line_magic('kql', magic_string)
    return results_frame

In [ ]:
# Select a LA workspace id from some demo environments
available_workspaces = {'ASIHuntOMSWorkspaceV4': '52b1ab41-869e-4138-9e40-2a4457f09bf0',
                        'ASIHuntOMSWorkspaceV5': '4ca7b24a-6e8f-4540-a8ce-1a80c2948c37',
                        'Contoso77':'802d39e1-9d70-404d-832c-2de5e2478eda', 
                        'MSTICLinux':'06dc719f-5dad-47e9-b5af-07d84a0bda4e',
                        'Rome ILDC - Detection E2E Tests Stage': '3eb61071-5dcd-4db3-94fa-0091a69b7359' }

alertId = ''
def select_workspace(x=''):
    global workspace_id
    workspace_id = available_workspaces[x]
     
selected_workspace = widgets.Select(options=available_workspaces.keys(), 
                              description='Select workspace :',
                              layout=Layout(width='50%', height='100px'),
                              style = {'description_width': 'initial'})
interactive(select_workspace, x=selected_workspace)

In [ ]:
# Kusto authentication
#%kql kusto://code().cluster('Ascprodus').database('ProdAlerts')
    
# Log Analytics authentication
%kql loganalytics://code().workspace(workspace_id)

# Sample office hunting and investigation

In [ ]:
# Look for unusual signins - eg associated with multiple IPs within short timeframe
unusual_signin_query = """
let timeRange=ago(120d);
let officeAuthentications = OfficeActivity
| where TimeGenerated >= timeRange
| where RecordType in ("AzureActiveDirectoryAccountLogon", "AzureActiveDirectoryStsLogon")
| extend UserAgent = extractjson("$[0].Value", ExtendedProperties, typeof(string))
| where Operation == "UserLoggedIn";
let lookupWindow = 1d;
let lookupBin = lookupWindow / 2.0; 
officeAuthentications | project-rename Start=TimeGenerated
| extend TimeKey = bin(Start, lookupBin)
| join kind = inner (
    officeAuthentications
    | project-rename End=TimeGenerated
    | extend TimeKey = range(bin(End - lookupWindow, lookupBin), bin(End, lookupBin), lookupBin)
    | mvexpand TimeKey to typeof(datetime)
) on UserAgent, TimeKey
| project timeSpan = End - Start, UserId, ClientIP , UserAgent , Start, End
| summarize dcount(ClientIP) by  UserAgent
| where dcount_ClientIP > 1
| join kind=inner (  
officeAuthentications
| summarize minTime=min(TimeGenerated), maxTime=max(TimeGenerated), dcount(ClientIP) by UserId, UserAgent
| project UserId, dcount_ClientIP , timeSpan = maxTime-minTime, UserAgent
| project UserId, distinctIPCount=dcount_ClientIP, timeSpan, UserAgent) on UserAgent
| project UserAgent, UserId, distinctIPCount
"""

unusual_signing_results = execute_kql_query(unusual_signin_query)
user_agents = unusual_signing_results.groupby('UserAgent')

selected_user_agent = list(user_agents.groups.keys())[0]

selected_user_id = user_agents.get_group(selected_user_agent).get('UserId').values[0]

def select_user_id(user_id):    
    global selected_user_id
    selected_user_id = user_id

def select_user_agent(user_agent):
    global selected_user_agent
    selected_user_agent = user_agent

    user_id_dropdown.options = user_agents.get_group(user_agent).get('UserId').values
    
user_agent_dropdown = widgets.Dropdown(options=user_agents.groups.keys())
init_user_agent_dropdown = user_agent_dropdown.value
user_id_dropdown = widgets.Dropdown(options=user_agents.get_group(init_user_agent_dropdown).get('UserId').values)

interactive_user_id = widgets.interactive(select_user_id, user_id=user_id_dropdown)
interactive_user_agent = widgets.interactive(select_user_agent, user_agent=user_agent_dropdown)

left_box = VBox([user_agent_dropdown])
right_box = VBox([user_id_dropdown])
HBox([left_box, right_box])

In [ ]:
# the query above returns counts for UA/UserId combinations
# So from there it would be good to pick one of the user agents and list all the signins for that UserID/UA
user_agent_signin_query = """
let timeRange=ago(120d);
let officeAuthentications =
OfficeActivity
| where TimeGenerated >= timeRange
| where RecordType in ("AzureActiveDirectoryAccountLogon", "AzureActiveDirectoryStsLogon")
| extend UserAgent = extractjson("$[0].Value", ExtendedProperties, typeof(string))
| where Operation == "UserLoggedIn";
officeAuthentications
| where UserId == "{user_id}" 
| where UserAgent == "{user_agent}"
| project TimeGenerated , UserId, Operation , ClientIP , UserAgent 
""".format(user_id=selected_user_id, user_agent=selected_user_agent)

user_agent_signin_results = execute_kql_query(user_agent_signin_query)
display(user_agent_signin_results)

In [ ]:
def get_geo(ip_array):
    geo_list = []
    for ip in ip_array:
        geo_match = geo.get(ip)
        geo_result = {
            'City': geo_match.get('city', {}).get('names', {}).get('en', 'Unknown'), 
            'Country': geo_match.get('country', {}).get('names', {}).get('en', 'Unknown'),
            'Country Code': geo_match.get('country', {}).get('iso_code', 'Unknown'),
            'Latitude': geo_match.get('location', {}).get('latitude', 'Unknown'),
            'Longitude': geo_match.get('location', {}).get('longitude', 'Unknown'),
            'IP': ip
        }
        geo_list.append(geo_result)
    return pd.DataFrame(data=geo_list)

geo = geolite2.reader()

user_agent_signin_results_geo = user_agent_signin_results.merge(get_geo(user_agent_signin_results['ClientIP'].unique()), left_on='ClientIP', right_on='IP')

display(user_agent_signin_results_geo.groupby(['Country', 'ClientIP'])['ClientIP'].agg(['count']))
display(user_agent_signin_results_geo.groupby(['Country'])['ClientIP'].count().plot.bar())

In [ ]:
map = folium.Map(zoom_start=7,tiles=None)
map.add_tile_layer()

marker_cluster = MarkerCluster()

for index, row in user_agent_signin_results_geo.iterrows():
    marker = folium.Marker(
        location = [row['Latitude'], row['Longitude']],
        popup="{City}, {Country}<br>{IP}<br>{UserId}<br>{UserAgent}".format(**user_agent_signin_results_geo.loc[index]),
        tooltip="{City}, {Country}".format(**user_agent_signin_results_geo.loc[index])
    )
    marker_cluster.add_child(marker)
    
map.add_child(marker_cluster)

display(map)